In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import shutil
import time
sns.set()

In [2]:
def get_filenames(path='.', extension=None, pattern=None, identifiers=None, include_path=False):
   
    # retrieve all filenames from the directory
    filename_list = os.listdir(path)
    # keep all filenames with the proper extension
    if extension is not None:
        filename_list = [filename for filename in filename_list if
                         filename[-len(extension):] == extension]
    # keep all filenames that match the pattern
    if pattern is not None:
        filename_list = [filename for filename in filename_list if re.search(pattern, filename)]
    # keep all filenames that match the identifiers provided
    if identifiers is not None:
        storage_list = []
        try:
            for ident in identifiers:
                storage_list.extend([filename for filename in filename_list if str(ident) in filename])
        except TypeError:
            print(identifiers, 'is not a list, tuple, or otherwise iterable')
        else:
            filename_list = storage_list
    if include_path:
        filename_list = [path + filename for filename in filename_list]
    return filename_list

# gives me my list of flux arrays
def get_filevalues(filename_list): 
    
    list_fluxarrays = []
    list_classtype = []
    
    for i in range(len(quasardata)):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(quasardata[i])+ "",\
                       memmap = False ) as hdul:
            
            data = hdul['COADD'].data
            data1 = hdul['SPALL'].data
            
            flux_val = data.field("flux")
            list_fluxarrays.append(flux_val) 
            
            classtype = data1.field('CLASS')
            list_classtype.append(classtype)
            
            values = {'FLUX': list_fluxarrays, 'CLASS': list_classtype}
            
            hdul.close()
            del hdul['COADD'].data
            del hdul['PRIMARY'].data
            del hdul['SPALL'].data
            del hdul
            
    return values

def preprocessing_data(flux_list, label_list):
    
    fluxlen_list = [len(flux_list[i]) for i in range(len(flux_list))]
    
    cut_fluxlen_list = [i for i in fluxlen_list if i >= 4550]
    
    filtered_flux_list = np.array(flux_list)[np.array(cut_fluxlen_list)]
    
    filtered_labels_list = np.array(label_list)[np.array(cut_fluxlen_list)]
    
    fluxlen_same = [filtered_flux_list[i][:4550] for i in range(len(filtered_flux_list))]
    
    return fluxlen_same

def creating_input_tensor(samples, height, width, channels, flux_list, label_list):
    
    input_tensor = np.ones((samples, height, width, channels))
    
    fluxlen_same = preprocessing_data(flux_list, label_list)
    
    for i in range(samples):
        spec = fluxlen_same[i]
        input_tensor[i:0,:,0] = spec[:]
        
    return input_tensor

In [3]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
quasar_dict = get_filevalues(quasardata)

In [4]:
quasar_flux = quasar_dict['FLUX']
quasar_labels = quasar_dict['CLASS'] 

In [5]:
processed_quasarflux = preprocessing_data(quasar_flux, quasar_labels)
quasar_tensor = (7948,1, 4550, 1, quasar_flux, quasar_labels) 



In [12]:
np.shape(quasar_tensor)

(7948, 1, 4550, 1)


In [7]:
# get list of the lengths of the flux arrays
quasar_fluxlen_list = [len(quasar_flux[i]) for i in range(len(quasar_flux))]

# create list of all the flux lengths that are greater than 4550
cut_quasar_fluxlen_list = [i for i in quasar_fluxlen_list if i >= 4550]

# creates array of the star fluxs that have a length greater than 4550
filtered_quasar_flux_list = np.array(quasar_flux)[np.array(cut_quasar_fluxlen_list)]
print(len(filtered_quasar_flux_list))

# creates array of the star labels that correspond to 
filtered_quasar_labels_list = np.array(quasar_labels)[np.array(cut_quasar_fluxlen_list)]
print(len(filtered_quasar_labels_list))

# creates array of star fluxs that have all the same length ie len of 4550
quasar_fluxlen_same = [filtered_quasar_flux_list[i][:4550] for i in range(len(filtered_quasar_flux_list))]
quasarflux_same = quasar_fluxlen_same

7948
7948


In [8]:
quasar_tensor = np.ones((7948,1,4550,1))

for i in range(7948):
    spec = quasarflux_same[i]
    quasar_tensor[i,0,:,0] = spec[:]
    
print(np.shape(quasar_tensor))

(7948, 1, 4550, 1)


In [9]:
print((filtered_quasar_flux_list[0][100:110])) # why is the rounding different between these two?
print((quasar_tensor[0,0,100:110,0]))

[27.820717 24.073212 25.996601 27.042116 25.894186 27.190266 26.642279
 25.34961  27.78979  29.07426 ]
[27.82071686 24.07321167 25.9966011  27.04211617 25.89418602 27.19026566
 26.64227867 25.34960938 27.7897892  29.07426071]
